# 1. Bag of Words( BoW )

- Bag of Words는 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도( frequency )에만 집중하는 텍스트 데이터의 수치적 표현 방법( Local Representation )
- 갖고 있는 어떤 텍스트 문서에 있는 단어들을 가방에 전부 넣은 다음 가방을 흔들어 단어들을 섞고, 해당 문서 내에서 특정 단어가 n번 등장했다면, 가방에서 해당 특정 단어 n개가 있게 된다. 여기서 가방을 흔들어서 단어를 섞었기 때문에 ***순서***는 중요하지 않다.

- BoW 만드는 과정
    1. 각 단어에 대한 고유한 정수 인덱스를 부여한다.
    2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

In [1]:
import re # 정규 표현식( regular expression )
from konlpy.tag import Okt

In [2]:
okt = Okt()

In [3]:
# "정부가 발표하는 물가상승류과 소비자가 느끼는 물가상승률은 다르다"를
# BoW로 표현
# 정규 표현식을 이용한 '.'제거하는 clearning 작업
token = re.sub( "(\.)", "", 
    "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")

In [4]:
token = okt.morphs( token )
token

['정부', '가', '발표', '하는', '물가상승률', '과', '소비자', '가', '느끼는', '물가상승률', '은', '다르다']

In [5]:
word2index = {}
bow = []

for vocab in token:
    if vocab not in word2index.keys():
        # 최초 등장 단어에 대한 단어 사전 등록 및 초기값 부여
        word2index[ vocab ] = len( word2index )
        bow.insert( len( word2index ) - 1, 1 )
    else:
        # 단어 사전에 등록된 단어에 대한 빈도수 증가
        index = word2index.get( vocab )
        bow[ index ] = bow[ index ] + 1

In [6]:
# 문장에 대한 정수 인덱스
word2index

{'정부': 0,
 '가': 1,
 '발표': 2,
 '하는': 3,
 '물가상승률': 4,
 '과': 5,
 '소비자': 6,
 '느끼는': 7,
 '은': 8,
 '다르다': 9}

In [7]:
# BoW
bow

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

- BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법이다.
- 주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격의 문서인지를 판단하는 작업에 사용된다.
- 즉, 분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 쓰인다.
    - '달리기', '체력', '근력'과 같은 단어가 자주 등장하면 해당 문서를 체육 관련 문서로 분류할 수 있을 것이며, '미분', '방정식', '부등식'과 같은 단어가 자주 등장한다면 수학 관련 문서로 분류할 수 있다.

#### CountVectorizer 클래스로 BoW 생성

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
corpus = [ 'you know I want your love. because I love you' ]

In [10]:
vector = CountVectorizer()

In [11]:
print( vector.fit_transform( corpus ).toarray() ) # BoW, 각 단어의 빈도수

[[1 1 2 1 2 1]]


In [12]:
print( vector.vocabulary_ ) # 각 단어의 인덱스

{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


#### 불용어를 제거한 BoW 생성

In [20]:
# 1) 사용자 정의 불용어 사용
text = [ "Family is not an important thing. It's everything." ]

In [21]:
vector = CountVectorizer( stop_words = [ 'the', 'a', 'an', 'is', 'not' ] )

In [23]:
vector.fit_transform( text ).toarray()

array([[1, 1, 1, 1, 1]], dtype=int64)

In [24]:
vector.vocabulary_

{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}

In [25]:
# 2) CountVectorizer 제공 불용어 사용
vector = CountVectorizer( stop_words = "english" )

In [26]:
vector.fit_transform( text ).toarray()

array([[1, 1, 1]], dtype=int64)

In [27]:
vector.vocabulary_

{'family': 0, 'important': 1, 'thing': 2}

In [28]:
# NLTK 제공 불용어 사용
from nltk.corpus import stopwords

In [29]:
stop_words = stopwords.words( 'english' )
vector = CountVectorizer( stop_words = stop_words )

In [30]:
vector.fit_transform( text ).toarray()

array([[1, 1, 1, 1]], dtype=int64)

In [31]:
vector.vocabulary_

{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}

# 2. 문서 단어 행렬( Document-Term Matrix, DTM )

- 문서 단어 행렬( Document-Term Matrix, DTM )은 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것이다.
- 즉, 각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 생각할 수 있으며, BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어이다.

#### DTM 예

문서 1 : 먹고 싶은 사과  
문서 2 : 먹고 싶은 바나나  
문서 3 : 길고 노란 바나나 바나나   
문서 4 : 저는 과일이 좋아요  

- 문서 1 ~ 문서 4에 대한 DTM

|-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|  
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|  
|문서 1|0|0|0|1|0|1|1|0|0|  
|문서 2|0|0|0|1|1|0|1|0|0|  
|문서 3|0|1|1|0|2|0|0|0|0|  
|문서 4|1|0|0|0|0|0|0|1|1|  

#### 문서 단어 행렬( Document-Term Matrix, DTM ) 한계

1) 희소 표현( Sparse representation )
    - 원-홧 벡터는 단어 집합의 크기가 벡터의 차원이 되고 대부분의 값이 0이 된다는 특징이 있다.
    - 이 특징은 공간적 낭비와 계산 리소스를 증가시킬 수 있다는 점에서 원-홧 벡터의 단점이 된다.
    - DTM도 원-홧 벡터와 동일한 단점을 갖는다.
    - DTM에서의 각 행은 문서 벡터를 의미하고, 문서 벡터의 차원은 원-홧 벡터와 마찬가지로 전체 단어 집합의 크기를 가진다.
    - 전체 코퍼스가 방대한 데이터라면 문서 벡터의 차원은 수백만의 차원을 가질 수 있고, 또한 많은 문서 벡터가 대부분의 값이 0을 가질 수도 있다.
    - 원-홧 벡터나 DTM과 같은 대부분의 값이 0인 표현을 희소 벡터( sparse vector ) 또는 희소 행렬( sparse matrix )라 한다.
    - 희소 벡터는 많은 양의 저장 공간과 계산을 위한 리소스가 필요하다.
    - 따라서 텍스트 전처리시에 단어 집합의 크기를 줄이는 일은 BoW / DTM 표현은 사용하는 모델에서는 매우 중요하다.

2) 단순 빈도 수 기반 접근
    - 여러 문서에 등장하는 모든 단어에 대해서 빈도 표기를 하는 방법은 때로 한계를 가지고 있다.
    - 예로 영어에 대해서 DTM을 만들었을 때, 불용어인 the는 어떤 문서이든 자주 등장할 수 밖에 없다. 그런데 유사한 문서인지 비교하고 싶은 문서 1, 문서 2, 문서 3에서 동일하게 the가 빈도수가 높다고 해서 이 문서들이 유사한 문서라고 판단할 수 없다.
    - 각 문서에는 중요한 단어와 불필요한 단어들이 혼재되어 있다. 따라서 DTM에 불용어와 중요한 단어에 대한 구별 방법이 별도로 제공되지 않는다.

## TF-IDF( 단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency )

- TF-IDF( 단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency )는 단어의 빈도수와 역 문서 빈도( 문서의 빈도에 특정 식을 취한다.)를 사용하여 DTM내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- TF-IDF 만드는 과정
    1. DTM을 만든다.
    2. TF-IDF 가중치를 부여한다.
- TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업등에 쓰일 수 있다.

- TF-IDF는 TF와 IDF를 곱한 값을 의미한다.
- 문서를 d, 단어를 t, 문서의 총 개수를 n이라 표현할 때 TF, DF, IDF는 다음과 같이 정의한다.

1) tf( d, t ) : 특정 문서 d에서의 특정 단어 t의 등장 회수
    - TF는 DTM에서 단어들이 가진 값들이다. 
    - DTM이 각 문서에서의 각 단어의 등장 빈도를 나타내는 값이기 때문이다.
    
2) df( t ) : 특정 단어 t가 등장한 문서의 수
    - 여기서 특정 단어가 각 문서, 또는 문서들에서 몇 번 등장했는지는 관심가지지 않으며 오직 특정 단어 t가 등장한 문서의 수에만 관심을 가진다.
    
3) idf( d, t ) : df( t )에 반비례하는 수  
    $idf(d, t) = log(\frac{n}{1+df(t)})$
    - IDF는 DF의 역수를 취한다. 
    - log를 사용하지 않았을 때, IDF를 DF의 역수( $\frac{n}{df(t)}$ 식 )로 사용한다면 총 문서의 수 n이 커질 수록, IDF의 값은 기하급수적으로 커지게 된다. 그렇기 때문에 log를 사용한다.

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다.
- TF-IDF 값이 낮으면 중요도가 낮은 거이며, TF-IDF 값이 크면 중요도가 큰 것이다.

#### 파이썬으로 TF-IDF 구현

In [32]:
from math import log

import numpy as np
import pandas as pd

In [52]:
# TF-IDF를 테스트할 문서라 가정
docs = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]

In [53]:
vocab = list( set( w for doc in docs for w in doc.split() ) )
vocab

['좋아요', '사과', '길고', '과일이', '먹고', '바나나', '노란', '저는', '싶은']

In [54]:
vocab.sort()
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [55]:
# TF, DF, IDF 함수 정의
N = len( docs )

def tf( t, d ):
    return d.count( t )

def idf( t ):
    df = 0
    for doc in docs:
        df += t in doc
    
    return log( N / ( df + 1 ) )

def tfidf( t, d ):
    return tf( t, d ) * idf( t )

In [56]:
# TF 계산
result = []

for i in range( N ):
    result.append( [] )
    d = docs[ i ]
    for j in range( len( vocab ) ):
        t = vocab[ j ]
        result[ -1 ].append( tf( t, d ) )
        
tf_ = pd.DataFrame( result, columns = vocab )
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [57]:
# IDF( 역문서 빈도 ) 계산
result = []

for j in range( len( vocab ) ):
    t = vocab[ j ]
    result.append( idf( t ) )
    
idf_ = pd.DataFrame( result, index = vocab, columns = [ 'IDF' ] )
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [58]:
# TF-IDF 행렬 계산
result = []

for i in range( N ):
    result.append( [] )
    d = docs[ i ]
    for j in range( len( vocab ) ):
        t = vocab[ j ]
        result[ -1 ].append( tfidf( t, d ) )

tfidf_ = pd.DataFrame( result, columns = vocab )
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


#### 사이킷런을 이용한 DTM과 TF-IDF 구현

In [59]:
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do'
]

In [60]:
vector = CountVectorizer()

In [61]:
# DTM
print( vector.fit_transform( corpus ).toarray() )
print( vector.vocabulary_ )

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [63]:
tfidfv = TfidfVectorizer().fit( corpus )

In [64]:
# TF-IDF
print( tfidfv.transform( corpus ).toarray() )
print( tfidfv.vocabulary_ )

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


# 3. 순환신경망( Recurrent Neural Network, RNN )

- RNN( Recurrent Neural Network )은 시퀀스( Sequence ) 모델이다.
- 입력과 출력을 시퀀스 단위로 처리하는 모델이다.
    - 번역기를 예로 들면 입력은 번역하고자 하는 문장( 단어 시퀀스 )이고, 출력은 해당되는 번역된 문장 또는 단어 시퀀스이다.
- 시퀀스들을 처리하기 위해 고안된 모델들을 시퀀스 모델이라 하고, RNN은 Deep Learning에서 가장 기본적인 시퀀스 모델이다.

## 1. 순환 신경망( Recurrent Neural Network, RNN )

- 피드포워드 신경망( FFNN )은 전부 은닉층에서 활성화 함수를 지난 값은 오직 출력층 방향으로만 향하였다.
- RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서, 다시 은닉층 노드의 다음 계산으로 보내는 특징을 갖고 있다.

![Alt text]( rnn_image1_ver2.png )

- x는 입력층의 입력벡터, y는 출력층의 출력벡터, 편향 b도 입력으로 존재하지만 그림에서는 생략
- RNN에서 은닉층에서 활성화 함수를 통해 결과를 내보내는 역활을 하는 노드를 셀( cell )이라 한다.
- 셀은 이전의 값을 기억하려고 하는 일종의 메모리 역활을 수행하므로 이를 메모리 셀 또는 RNN 셀이라 표현한다.


- 은닉층의 메모리 셀을 각각의 시점( time step )에서 바로 이전 시점에서의 은닉층의 메모리 셀에서 나온 값을 자신의 입력으로 사용하는 재귀적 활동을 한다.
- 현재 시점을 변수 t로 표현한다.
- 현재 시점 t에서의 메모리 셀이 갖고있는 값은 과거의 메모리 셀들의 값에 영향을 받은 것임을 의미한다.


- 메모리 셀이 출력층 방향으로 또는 다음 시점 t + 1의 자신에게 보내는 값을 은닉 상태( hidden state )라고 한다. 즉, t 시점의 메모리 셀은 t - 1 시점의 메모리 셀이 보낸 은닉 상태값을 t 시점의 은닉 상태 계산을 위한 입력값으로 사용한다.

![Alt text]( rnn_image2_ver3.png )

- 위 그림은 RNN을 표현할 때 일반적으로 좌측과 같이 화살표로 사이클을 그려서 재귀 형태로 표현하기도 하고, 사이클을 그리는 화살표 대신 여러 시점으로 펼쳐서 표현하기도 한다.

- 피드 포워드 신경망에서는 뉴런이라는 단위를 사용하지만, RNN에서는 뉴런이라는 단위보다는 입력층과 출력층에서는 각각 입력 벡터, 출력 벡터라 하고, 은닉층에서는 은닉 상태라는 표현을 주로 사용한다.

![Alt text]( rnn_image2.5.png )

- 이 그림은 RNN을 뉴런 단위로 표현한 내용으로 입력 벡터 4차원, 은닉 상태 크기 2, 출력 벡터 2차원인 RNN이 시점이 2일 때의 모습을 표현한 그림이다.

![Alt text]( rnn_image3_ver2.png )

- RNN은 입력과 출력의 길이를 다르게 설계 할 수 있으므로 다양한 용도로 사용할 수 있다.

- 하나의 입력에 대해서 여러개의 출력( one-to-many )의 모델은 하나의 이미지 입력에 대해서 사진의 제목을 출력하는 이미지 캡셔닝( Image Captioning ) 작업에 사용할 수 있다.

![Alt text]( rnn_image3.5.png )

- 단어 시퀀스에 대해서 하나의 출력( many-to-one )을 하는 모델은 입력 문서가 긍정적인지 부정적인지를 판별하는 감성 분류( sentiment classification ), 또는 메일이 정상 메일인지 스팸 메일인지 판별하는 스팸 메일 분류( spam detection )에 사용할 수 있다.

![Alt text]( rnn_image3.7.png )

- 다 대 다( many-to-many )의 모델의 경우에는 입력 문장으로 부터 대답 문자을 출력하는 챗봇과 입력 문장으로부터 번역된 문장을 출력하는 번역기, 개체명 인식이나 품사 태깅 작업을 수행하는 경우에 사용할 수 있다.

- RNN 수식

![Alt text]( rnn_image7_ver2.png )

- 현재 시점 t에서의 은닉 상태값을 $h_{t}$라고 정의하고, 은닉층의 메모리 셀은 $h_{t}$를 계산하기 위해서 총 두 개의 가중치를 갖게 된다.
- 하나는 입력층에서 입력값을 위한 가중치 $W_{x}$이고, 하나는 이전 시점   t-1의 은닉 상태값인 $h_{t-1}$을 위한 가중치 $W_{h}$이다.


은닉층 : $h_{t} = tanh(W_{x} x_{t} + W_{h}h_{t−1} + b)$    
출력층 : $y_{t} = f(W_{y}h_{t} + b)$  
단, $f$는 비선형 활성화 함수중 하나이다.

- RNN은 은닉층 연산을 벡터와 행렬 연산으로 이해할 수 있다.
- 자연어 처리에서 RNN의 입력 $x_{t}$는 대부분의 경우에 단어 벡터로 간주할 수 있는데, 단어 벡터의 차원을 $d$라고 하고, 은닉 상태의 크기를 $D_{h}$라고 하였을 때 각 벡터와 행렬의 크기는 다음과 같다.

$x_{t}$ : $(d x 1)$  
$W_{x}$ : $(D_{h} x d)$  
$W_{h}$ : $(D_{h} x D_{h})$  
$h_{t-1}$ : $(D_{h} x 1 )$  
$b$ : $(D_{h} x 1 )$

- 배치 크기가 1이고, $d$와 $D_{h}$ 두 값 모두를 4로 가정하였을 때, RNN의 은닉층 연산은 다음과 같다.

![Alt text]( rnn_images4-5.png )

- $h_{t}$를 계산하기 위한 활성화 함수로는 주로 하이퍼볼릭탄젠트 함수(tanh)가 사용되지만, ReLU로 바꾸 사용하는 시도도 있다.
- 각각의 가중치 $W_{x}, W_{h}, W_{y}$의 값은 모두 시점에서 값을 동일하게 공유한다. 만약 은닉층이 2개 이상일 경우에는 은닉층 2개의 가중치는 서로 다르다.
- 출력층은 결과값인 $y_{t}$를 계산하기 위한 활성화 함수로는 상황에 따라 다르지만 예로 이진 분류를 해야하는 경우라면 시그모이드 함수를 사용할 수 있고 다양한 카테고리 중에서 선택해야하는 문제라면 소프트맥스 함수를 사용하게 된다.

### BPTT( Backpropagation through time )

- RNN도 다른 인공 신경망과 마찬가지로 역전파를 통해 학습을 진행한다.
- FFNN의 역전파와 다른 점이 있다면 RNN은 전체 시점에 대해서 네트워크를 펼친 다음에 역전파를 사용하며 모든 시점에 대해서 가중치를 공유하고 있다는 점이다.
- RNN의 이러한 역전파 과정을 BPTT( Backpropagation through time, 시간에 따른 역전파 )라고 부른다.

### RNN을 이용한 텍스트 생성( Text Generation Using RNN )

In [65]:
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [66]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

#### 데이터 전처리 및 EDA

In [71]:
t = Tokenizer()
t.fit_on_texts( [ text ] )
vocab_size = len( t.word_index ) + 1
print( '단어 집합의 크기 : {}'.format( vocab_size ) )

단어 집합의 크기 : 12


In [72]:
# 정수 인덱스
print( t.word_index )

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [73]:
# 훈련 데이터 생성
sequences = list()

for line in text.split( '\n' ):
    encoded = t.texts_to_sequences( [ line ] )[ 0 ]
    for i in range( 1, len( encoded ) ):
        sequence = encoded[ :i + 1 ]
        sequences.append( sequence )
        
print( '학습에 사용할 샘플의 개수 : {}'.format( len( sequences ) ) )

학습에 사용할 샘플의 개수 : 11


In [74]:
print( sequences )

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [75]:
max_len = max( len( l ) for l in sequences )
print( '샘플의 최대 길이 : {}'.format( max_len ) )

샘플의 최대 길이 : 6


In [76]:
sequences = pad_sequences( sequences, maxlen = max_len, padding = 'pre' )

In [77]:
print( sequences )

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [78]:
# 입력 데이터와 레이블 분리
sequences = np.array( sequences )
X = sequences[ :, :-1 ]
y = sequences[ :, -1 ]

In [79]:
print( X ) # 입력 데이터

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [80]:
print( y ) # 레이블

[ 3  1  4  5  1  7  1  9 10  1 11]


In [83]:
y = to_categorical( y, num_classes = vocab_size )

In [84]:
print( y )

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


#### 모델 설계

In [87]:
model = Sequential()
model.add( Embedding( vocab_size, 10, input_length = max_len - 1 ) )
model.add( SimpleRNN( 32 ) )
model.add( Dense( vocab_size, activation = 'softmax' ) )

In [89]:
model.compile( loss = 'categorical_crossentropy', optimizer = 'adam',
               metrics = [ 'accuracy' ] )
model.fit( X, y, epochs = 200, verbose = 2 )

Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.3340 - accuracy: 0.3636
Epoch 2/200
11/11 - 0s - loss: 2.3156 - accuracy: 0.3636
Epoch 3/200
11/11 - 0s - loss: 2.2965 - accuracy: 0.3636
Epoch 4/200
11/11 - 0s - loss: 2.2766 - accuracy: 0.3636
Epoch 5/200
11/11 - 0s - loss: 2.2560 - accuracy: 0.3636
Epoch 6/200
11/11 - 0s - loss: 2.2348 - accuracy: 0.3636
Epoch 7/200
11/11 - 0s - loss: 2.2129 - accuracy: 0.3636
Epoch 8/200
11/11 - 0s - loss: 2.1904 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.1674 - accuracy: 0.3636
Epoch 10/200
11/11 - 0s - loss: 2.1441 - accuracy: 0.3636
Epoch 11/200
11/11 - 0s - loss: 2.1205 - accuracy: 0.3636
Epoch 12/200
11/11 - 0s - loss: 2.0968 - accuracy: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.0733 - accuracy: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.0501 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.0275 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.0057 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 1.9849 - accuracy: 0.3636
Epo

Epoch 142/200
11/11 - 0s - loss: 0.2848 - accuracy: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.2789 - accuracy: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.2731 - accuracy: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2675 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2620 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2566 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2513 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2461 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2411 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2361 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2313 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2266 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2220 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.2175 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2130 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2087 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2045 - accuracy: 1.00

#### Text 생성

In [90]:
# 인수 : model - 모델, t - 토크나이거, 
#        current_word - 현재 단어, n - 예측을 반복할 횟수
def sentence_generation( model, t, current_word, n ):
    init_word = current_word
    sentence = ''
    
    for _ in range( n ):
        encoded = t.texts_to_sequences( [ current_word ] )[ 0 ]
        encoded = pad_sequences( [ encoded ], maxlen = 5, 
                                padding = 'pre' )
        result = model.predict_classes( encoded, verbose = 0 ) 
        # 입력한 X(현재단어)에 대한 y(예측한 단어)를 result에 저장
        
        for word, index in t.word_index.items():
            if index == result: # 예측할 단어와 동일한 단어가 있다면
                break;
        current_word = current_word + ' ' + word
        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    sentence = init_word + sentence
    
    return sentence

In [91]:
print( sentence_generation( model, t, '경마장에', 4 ) )

경마장에 있는 말이 뛰고 있다


In [92]:
print( sentence_generation( model, t, '그의', 2 ) )

그의 말이 법이다


In [93]:
print( sentence_generation( model, t, '가는', 5 ) )

가는 말이 고와야 오는 말이 곱다
